### <font color='#4285f4'>Overview</font>

This notebook will generate menu data and vector embeddings for the menu items and ingredients tables.  This data is already loaded by the intialize stored procedure.  You can drop the table to run those cells.

The Search Embeddings and Menu Search with re-ranking will search the data using the embeddings.  The Re-ranking will combine two different searches into one and then re-rank the search results so we get the best menu options at the top of the webpage showing the menu items.

Cost:
* Low: Gemini, BigQuery
* Medium: Remember to stop your Colab Enterprise Notebook Runtime

Author:
* Adam Paternostro

### <font color='#4285f4'>License</font>

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### <font color='#4285f4'>Pip Installs</font>

In [ ]:
# PIP Installs
import sys

# https://PLACEHOLDER.com/index.html
#!{sys.executable} -m pip install PLACEHOLDER

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "${bigquery_location}"
region = "${region}"
location = "${location}"
dataset_name = "${bigquery_chocolate_ai_dataset}"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Helper Methods</font>

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str, project_id=None) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  # Required by some API calls
  if project_id != None:
    headers["x-goog-user-project"] = project_id

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### RetryCondition (for retrying LLM calls)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### Gemini LLM (Pro 1.0 , Pro 1.5, Flash 2.0)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-2.0-flash-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-2.0-flash"

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-2.0-flash":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Vertex Re-Ranking API

In [ ]:
def vertex_ai_reranking(project_id, query, records):
  """Re-ranks search results"""


  url = f"https://discoveryengine.googleapis.com/v1/projects/{project_id}/locations/global/rankingConfigs/default_ranking_config:rank"

  request_body = {
      "model": "semantic-ranker-512@latest",
      "query": query,
      "records": records
      }

  print(request_body)

  json_result = restAPIHelper(url, "POST", request_body, project_id)

  return json_result

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

### <font color='#4285f4'>BigQuery Tables (for Generated Data)</font>

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create an ingredients table which will reside in Oracle
-- Drop the table to re-run the GenAI in this notebooks
-- DROP TABLE IF EXISTS `chocolate_ai.generated_ingredients`;
----------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `chocolate_ai.generated_ingredients` AS
SELECT GENERATE_UUID()      AS ingredient_id,
       menu_id,
       menu_name,
       menu_description,

       CAST(NULL AS STRING) AS ingredient_prompt,
       CAST(NULL AS STRING) AS ingredient_information,
       CAST(NULL AS STRING) AS ingredient_explanation,

       CAST(NULL AS STRING) AS allergy_prompt,
       CAST(NULL AS STRING) AS allergy_information,
       CAST(NULL AS STRING) AS allergy_explanation,

  FROM `chocolate_ai.menu`;

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create an inventory table which will reside in Oracle
-- This will show low inventory on the website
----------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `chocolate_ai.generated_inventory` AS
SELECT GENERATE_UUID() AS inventory_id,
       menu_id,
       CAST(ROUND(RAND() * (25 - 1)) AS INT64) AS inventory_quantity,
  FROM `chocolate_ai.generated_ingredients`;


In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create customer allergy table that contains "complex allergy sentenance" which will reside in Oracle
-- This will show the customer allery preferences on the website
----------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `chocolate_ai.generated_customer_allergy_information` AS
SELECT GENERATE_UUID() AS customer_allergy_information_id,
       customer_id,
       CAST(NULL AS STRING) AS allergy_information_prompt,
       CAST(NULL AS STRING) AS allergy_information,
       CAST(NULL AS STRING) AS allergy_information_explanation,
  FROM `chocolate_ai.customer`;

### <font color='#4285f4'>Generate Data</font>

#### Create ingredients based upon menu items

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View the data
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_ingredients` ORDER BY menu_id LIMIT 10;

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create a prompt that will create a list of ingredients based upon each menu item (name and description)
----------------------------------------------------------------------------------------------------------
UPDATE `chocolate_ai.generated_ingredients`
  SET ingredient_prompt = CONCAT("""You are a master chocolatier and expert at understanding the ingredients needed to create finely crafted chocolates.  Read the below description of our chocolate and think through the ingredients step by step.  The goal is to be able to determine allergy information so keep this in mind, but do not generate the allergy information at this time.

<menu-name>""",
menu_name,
"</menu-name>",
"<menu-description>",
menu_description,
"</menu-description>")
WHERE ingredient_prompt IS NULL;


In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View the generated prompts
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_ingredients` ORDER BY menu_id;

In [ ]:
%%bigquery bq_dataframe

----------------------------------------------------------------------------------------------------------
-- Select the BigQuery data into a dataframe so we can loop over in Python code
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_ingredients` WHERE ingredient_information IS NULL;

In [ ]:
##########################################################################################################
# For each item in the dataframe of ingredients call Vertex AI and execute the prompt
# Save the results into the ingredients table
# We could also used the BigQuery (BQML) ML.GENERATE_TEXT
##########################################################################################################

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "ingredient_information" : "text",
#    "ingredient_explanation" : "text"
#  }

response_schema = {
  "type": "object",
  "required": [
    "ingredient_information",
    "ingredient_explanation"
  ],
  "properties": {
    "ingredient_information": {
      "type": "string"
    },
    "ingredient_explanation": {
      "type": "string"
    }
  }
}

for index, row in bq_dataframe.iterrows():
  menu_id = row["menu_id"]
  print(f"menu id: {menu_id}")
  prompt = row["ingredient_prompt"]

  # Use LLM to generate data
  llm_response = GeminiLLM(prompt, response_schema=response_schema, temperature=.5)

  # Parse response (we know the JSON since we passed it to our LLM)
  llm_json_response = json.loads(llm_response)
  print(json.dumps(llm_json_response, indent=2))
  ingredient_information = llm_json_response["ingredient_information"]
  ingredient_explanation = llm_json_response["ingredient_explanation"]

  # Update BigQuery
  sql = f'''UPDATE `chocolate_ai.generated_ingredients`
  SET ingredient_information = """{ingredient_information}""",
      ingredient_explanation = """{ingredient_explanation}"""
  WHERE menu_id = {menu_id}'''

  print(f"sql: {sql}")
  RunQuery(sql)

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- See our results from Gemini
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_ingredients` WHERE ingredient_information IS NOT NULL ORDER BY menu_id;

#### Create allergy information based upon the menu ingredients

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create a prompt that will take the ingredients and extract allergy information.
-- This is done to show artifical data generation and in real life you would use the actual ingredients.
----------------------------------------------------------------------------------------------------------
UPDATE `chocolate_ai.generated_ingredients`
  SET allergy_prompt = CONCAT("""Role: You are an expert food scientist specializing in chocolate and allergen identification.  You possess an encyclopedic knowledge of ingredients, including their common names, scientific names, and derived components. You are meticulous and never miss a potential allergen.

Objective: Analyze the provided chocolate ingredients and identify *ALL* potential allergens based on a comprehensive allergy list.  This includes direct matches, semantic matches (synonyms and related terms), and *implicit* allergens based on common chocolate-making practices. Assume nothing is safe unless proven otherwise.

Output:  Generate a JSON object with a single field: `allergy_information`.  .

Output: The value of this field should be a comma-separated string containing all identified allergens from the provided list.  If no allergens are found, the value should be an empty string ("")

Input Data:

<allergy_list>
{
   "allergy": [
     "Milk", "Dairy", "Lactose", "Casein", "Whey",
     "Soy", "Soybean Oil", "Lecithin",
     "Tree Nuts", "Almonds", "Hazelnuts", "Walnuts", "Pecans", "Cashews", "Pistachios", "Macadamia Nuts", "Brazil Nuts",
     "Peanuts",
     "Wheat", "Gluten", "Oats", "Barley", "Rye",
     "Eggs",
     "Sesame",
     "Coconut",
     "Sunflower Seeds", "Sunflower Oil",
     "Mustard",
     "Sulfites",
     "Corn", "High Fructose Corn Syrup", "Dextrose", "Maltodextrin", "Corn Starch",
     "Food Dyes", "Red 40", "Yellow 5", "Blue 1",
     "Chocolate", "Theobromine", "Cocoa Mass", "Cocoa Butter",
     "Cinnamon",
     "Vanilla",
     "Artificial Sweeteners", "Aspartame", "Sucralose", "Saccharin",
     "Benzoates", "Sodium Benzoate",
     "Sorbates", "Potassium Sorbate",
     "Rice",
     "Gelatin",
     "Honey", "Royal Jelly", "Propolis", "Bee Pollen",
     "Shellac",
     "Palm Oil",
     "Canola Oil",
     "Citric Acid",
     "MSG (Monosodium Glutamate)",
     "Xanthan Gum",
     "Guar Gum",
     "Carrageenan",
     "Inulin",
     "Fructose", "Galactose",
     "Mannitol", "Sorbitol", "Xylitol", "Erythritol",
     "Isomalt",
     "Stevia",
     "Monk Fruit Extract",
     "Agave",
     "Tapioca",
     "Potato Starch",
     "Modified Food Starch",
     "Dextrin",
     "Cellulose",
     "Pectin",
     "Agar-Agar",
     "Locust Bean Gum",
     "Tara Gum",
     "Acacia Gum",
     "Tragacanth Gum",
     "Karaya Gum",
     "Gellan Gum",
     "Konjac Gum",
     "Rennet",
     "Annatto",
     "Carmine", "Cochineal Extract",
     "Beetroot Red",
     "Turmeric",
     "Saffron",
     "Paprika",
     "Spirulina",
     "Chlorella",
     "Algae",
     "Quinoa",
     "Amaranth",
     "Buckwheat",
     "Chia Seeds",
     "Flax Seeds",
     "Hemp Seeds",
     "Safflower Oil",
     "Grape Seed Oil",
     "Avocado Oil",
     "Olive Oil",
     "Coffee",
     "Tea",
     "Herbs", "Mint", "Lavender", "Rosemary",
     "Spices", "Clove", "Nutmeg", "Ginger",
     "Fruits", "Strawberries", "Raspberries", "Blackberries", "Blueberries",
     "Citrus Fruits", "Orange", "Lemon", "Lime", "Grapefruit",
     "Nightshades", "Tomato", "Potato", "Peppers", "Eggplant",
     "Legumes", "Beans", "Lentils", "Peas",
     "Seeds", "Poppy Seeds", "Pumpkin Seeds", "Squash Seeds"
   ]
 }
</allergy_list>
<ingredient_information>""",
ingredient_information,
"</ingredient_information>",
"<ingredient_explanation>",
ingredient_explanation,
"</ingredient_explanation")
WHERE allergy_prompt IS NULL;

In [ ]:
%%bigquery bq_dataframe

----------------------------------------------------------------------------------------------------------
-- Place the prompt for allergies into a dataframe so we can loop over in Python code
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_ingredients` WHERE ingredient_information IS NOT NULL AND allergy_information IS NULL;

In [ ]:
##########################################################################################################
# Run each allergy prompt and update the table
##########################################################################################################

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "allergy_information" : "text",
#    "allergy_explanation" : "text"
#  }

response_schema = {
  "type": "object",
  "required": [
    "allergy_information",
    "allergy_explanation"
  ],
  "properties": {
    "allergy_information": {
      "type": "string"
    },
    "allergy_explanation": {
      "type": "string"
    }
  }
}

for index, row in bq_dataframe.iterrows():
  menu_id = row["menu_id"]
  print(f"menu id: {menu_id}")
  prompt = row["allergy_prompt"]

  # Use LLM to generate data
  llm_response = GeminiLLM(prompt, response_schema=response_schema, temperature=.5)

  # Parse response (we know the JSON since we passed it to our LLM)
  llm_json_response = json.loads(llm_response)
  print(json.dumps(llm_json_response, indent=2))
  allergy_information = llm_json_response["allergy_information"]
  allergy_explanation = llm_json_response["allergy_explanation"]

  # Update BigQuery
  sql = f'''UPDATE `chocolate_ai.generated_ingredients`
  SET allergy_information = """{allergy_information}""",
      allergy_explanation = """{allergy_explanation}"""
  WHERE menu_id = {menu_id}'''

  print(f"sql: {sql}")
  RunQuery(sql)

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View our completed "generated ingredients" table
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_ingredients` WHERE allergy_information IS NOT NULL ORDER BY menu_id;

#### Create customer allergies as complex sentences

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------
-- Create a prompt that will take the ingredients and extract allergy information.
-- This is done to show artifical data generation and in real life you would use the actual ingredients.
----------------------------------------------------------------------------------------------------------
UPDATE `chocolate_ai.generated_customer_allergy_information`
  SET allergy_information_prompt = CONCAT("""Role: You are a customer and need to set your allergy information so you do not purchase items you are allergic to.
 You are shopping at a chocolate shop that sells handmade specialty chocolates and serves various coffees.
 You are an expert food scientist specializing in chocolate and allergen identification.
 You possess an encyclopedic knowledge of ingredients, including their common names, scientific names, and derived components.

Objective: Generate 2 to 4 sentences describing your allergies.  This is for a demo and we want a large variety of words with similar meaning.  This sentence will be processed
by an LLM to extract keywords, removing all the noise.  The below allergy list are the allergies we have identified in our chocolate, so you can use this for reference as
as well as make up other allergies which will cause a non-match (which is good.)

Example: Severe nut allergy, especially hazelnuts and almonds. Also, avoid any chocolate containing traces of soy lecithin. I have had anaphylactic reactions in the past. Please
ensure all preparation surfaces are thoroughly cleaned. I also have a sensitivity to artificial sweeteners, particularly aspartame. I would prefer organic chocolate if possible.

Output: Generate a JSON object with a single field: `allergy_information`.

Output: The value of this field should be English sentences.  If no allergens are generated, the value should be an empty string ("")

Input Data:
<allergy_list>
{
   "allergy": [
     "Milk", "Dairy", "Lactose", "Casein", "Whey",
     "Soy", "Soybean Oil", "Lecithin",
     "Tree Nuts", "Almonds", "Hazelnuts", "Walnuts", "Pecans", "Cashews", "Pistachios", "Macadamia Nuts", "Brazil Nuts",
     "Peanuts",
     "Wheat", "Gluten", "Oats", "Barley", "Rye",
     "Eggs",
     "Sesame",
     "Coconut",
     "Sunflower Seeds", "Sunflower Oil",
     "Mustard",
     "Sulfites",
     "Corn", "High Fructose Corn Syrup", "Dextrose", "Maltodextrin", "Corn Starch",
     "Food Dyes", "Red 40", "Yellow 5", "Blue 1",
     "Chocolate", "Theobromine", "Cocoa Mass", "Cocoa Butter",
     "Cinnamon",
     "Vanilla",
     "Artificial Sweeteners", "Aspartame", "Sucralose", "Saccharin",
     "Benzoates", "Sodium Benzoate",
     "Sorbates", "Potassium Sorbate",
     "Rice",
     "Gelatin",
     "Honey", "Royal Jelly", "Propolis", "Bee Pollen",
     "Shellac",
     "Palm Oil",
     "Canola Oil",
     "Citric Acid",
     "MSG (Monosodium Glutamate)",
     "Xanthan Gum",
     "Guar Gum",
     "Carrageenan",
     "Inulin",
     "Fructose", "Galactose",
     "Mannitol", "Sorbitol", "Xylitol", "Erythritol",
     "Isomalt",
     "Stevia",
     "Monk Fruit Extract",
     "Agave",
     "Tapioca",
     "Potato Starch",
     "Modified Food Starch",
     "Dextrin",
     "Cellulose",
     "Pectin",
     "Agar-Agar",
     "Locust Bean Gum",
     "Tara Gum",
     "Acacia Gum",
     "Tragacanth Gum",
     "Karaya Gum",
     "Gellan Gum",
     "Konjac Gum",
     "Rennet",
     "Annatto",
     "Carmine", "Cochineal Extract",
     "Beetroot Red",
     "Turmeric",
     "Saffron",
     "Paprika",
     "Spirulina",
     "Chlorella",
     "Algae",
     "Quinoa",
     "Amaranth",
     "Buckwheat",
     "Chia Seeds",
     "Flax Seeds",
     "Hemp Seeds",
     "Safflower Oil",
     "Grape Seed Oil",
     "Avocado Oil",
     "Olive Oil",
     "Coffee",
     "Tea",
     "Herbs", "Mint", "Lavender", "Rosemary",
     "Spices", "Clove", "Nutmeg", "Ginger",
     "Fruits", "Strawberries", "Raspberries", "Blackberries", "Blueberries",
     "Citrus Fruits", "Orange", "Lemon", "Lime", "Grapefruit",
     "Nightshades", "Tomato", "Potato", "Peppers", "Eggplant",
     "Legumes", "Beans", "Lentils", "Peas",
     "Seeds", "Poppy Seeds", "Pumpkin Seeds", "Squash Seeds"
   ]
 }
</allergy_list>""")
WHERE allergy_information_prompt IS NULL;

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View the generated prompts
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_customer_allergy_information` WHERE allergy_information_prompt IS NOT NULL;

In [ ]:
%%bigquery bq_dataframe

----------------------------------------------------------------------------------------------------------
-- Place the prompt for allergies into a dataframe so we can loop over in Python code
-- NOTE: Change the LIMIT of 11000 if you want to test just a few rows
----------------------------------------------------------------------------------------------------------
SELECT *
  FROM `chocolate_ai.generated_customer_allergy_information`
 WHERE allergy_information_prompt IS NOT NULL
   AND allergy_information IS NULL
 ORDER BY customer_id LIMIT 11000;

In [ ]:
##########################################################################################################
# Run each customer allergy prompt and update the table
##########################################################################################################

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "allergy_information" : "text",
#    "allergy_information_explanation" : "text"
#  }

response_schema = {
  "type": "object",
  "required": [
    "allergy_information",
    "allergy_information_explanation"
  ],
  "properties": {
    "allergy_information": {
      "type": "string"
    },
    "allergy_information_explanation": {
      "type": "string"
    }
  }
}

for index, row in bq_dataframe.iterrows():
  customer_id = row["customer_id"]
  print(f"customer_id id: {customer_id}")
  prompt = row["allergy_information_prompt"]

  # Use LLM to generate data
  success = False
  while success == False:
    llm_response = GeminiLLM(prompt, response_schema=response_schema, temperature=.5)

    # Parse response (we know the JSON since we passed it to our LLM)
    llm_json_response = json.loads(llm_response)
    #print(json.dumps(llm_json_response, indent=2))
    allergy_information = llm_json_response["allergy_information"]
    allergy_information_explanation = llm_json_response["allergy_information_explanation"]

    # Update BigQuery
    sql = f'''UPDATE `chocolate_ai.generated_customer_allergy_information`
    SET allergy_information = """{allergy_information}""",
        allergy_information_explanation = """{allergy_information_explanation}"""
    WHERE customer_id = {customer_id}'''

    #print(f"sql: {sql}")
    try:
      RunQuery(sql)
      success = True
    except:
      print("Retrying")

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View our completed "generated customer allergy information" table
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.generated_customer_allergy_information` WHERE allergy_information IS NOT NULL ORDER BY customer_id DESC LIMIT 25;

### <font color='#4285f4'>BigQuery Tables (Final tables from the Generated Data tables)</font>

#### Create Final Tables from the Generated Data

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create the "Oracle tables" in this BigQuery dataset
-- These would be copied to Oracle (by hand)
----------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `chocolate_ai.oracle_customer` AS
SELECT *
  FROM `chocolate_ai.generated_customer_allergy_information`;

CREATE TABLE IF NOT EXISTS `chocolate_ai.oracle_ingredients` AS
SELECT *
  FROM `chocolate_ai.generated_ingredients`;

CREATE TABLE IF NOT EXISTS `chocolate_ai.oracle_inventory` AS
SELECT *
  FROM `chocolate_ai.generated_inventory`;

### <font color='#4285f4'>Create Embeddings</font>

#### Create the text embedding model so we can call text-embedding-005 directly from SQL

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Add a reference to text embedding 005 from vertex
----------------------------------------------------------------------------------------------------------
CREATE MODEL IF NOT EXISTS `chocolate_ai.textembedding_model`
  REMOTE WITH CONNECTION `us.vertex-ai`
  OPTIONS (endpoint = 'text-embedding-005');

#### Create the Menu Embedding **Allery** table which will hold each menu item and associated allery

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create a new table for the ingredients allergy (search) text embeddings
-- Split each allergy information by the comman and embedded each (do not do the whole string)
-- We want to match each allergy not the entire allergy string, so we will split and embed each one seperately.
----------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `chocolate_ai.menu_embeddings_allergy` AS
WITH split_allergies AS
(
  SELECT menu_id,
         menu_name,
         menu_description,
         SPLIT(LOWER(allergy_information), ',') AS allergy_array
    FROM `chocolate_ai.generated_ingredients`
),
allergies AS
(
  SELECT menu_id,
         menu_name,
         menu_description,
         allergy
    FROM split_allergies
         JOIN UNNEST(allergy_array) AS allergy
)
SELECT menu_id,
       menu_name,
       menu_description,
       allergy,
       ml_generate_embedding_result AS menu_allergy_embedding
  FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                            (SELECT menu_id,
                                    menu_name,
                                    menu_description,
                                    TRIM(LOWER(allergy)) AS allergy,
                                    TRIM(LOWER(allergy)) AS content  -- make lowercase
                               FROM allergies),
                             STRUCT(TRUE AS flatten_json_output,
                                    'SEMANTIC_SIMILARITY' as task_type,
                                    768 AS output_dimensionality));

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View the count.  We have each allergy embeddded seperately.
----------------------------------------------------------------------------------------------------------
SELECT COUNT(*) FROM `chocolate_ai.menu_embeddings_allergy`;

In [ ]:
%%bigquery
SELECT * FROM `chocolate_ai.menu_allergy_embeddings` ORDER BY menu_id LIMIT 25;

#### Create the Menu Embedding **Name** table which will hold each menu item and associated name

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Create a new table for the menu description (search) text embeddings
-- Embedding the title and seperately embed the description
----------------------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS `chocolate_ai.menu_embeddings_name` AS
SELECT menu_id,
       menu_name,
       menu_description,
       ml_generate_embedding_result AS menu_name_embedding
  FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                            (SELECT menu_id,
                                    menu_name,
                                    menu_description,
                                    LOWER(menu_name) AS content  -- make lowercase
                               FROM `chocolate_ai.menu`),
                             STRUCT(TRUE AS flatten_json_output,
                                    'SEMANTIC_SIMILARITY' as task_type,
                                    768 AS output_dimensionality));

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View the menu embedding table
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.menu_embeddings_name` ORDER BY menu_id LIMIT 5;

#### Create the Menu Embedding **Description** table which will hold each menu item and associated description

In [ ]:
%%bigquery
----------------------------------------------------------------------------------------------------------
-- Create a new table for the menu description (search) text embeddings
-- Embedding the title and seperately embed the description
----------------------------------------------------------------------------------------------------------

CREATE TABLE IF NOT EXISTS `chocolate_ai.menu_embeddings_description` AS
SELECT menu_id,
       menu_name,
       menu_description,
       ml_generate_embedding_result AS menu_description_embedding
  FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                            (SELECT menu_id,
                                    menu_name,
                                    menu_description,
                                    LOWER(menu_description) AS content  -- make lowercase
                               FROM `chocolate_ai.menu`),
                             STRUCT(TRUE AS flatten_json_output,
                                    'SEMANTIC_SIMILARITY' as task_type,
                                    768 AS output_dimensionality));

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- View the menu description embeddings.
----------------------------------------------------------------------------------------------------------
SELECT * FROM `chocolate_ai.menu_embeddings_description` ORDER BY menu_id LIMIT 5;

### <font color='#4285f4'>Search Embeddings</font>

#### Basic search for a single term

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- We can do simple searchs like "milk" or "lactose"
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       COUNT(base.menu_id) AS menu_count,
       --base.menu_name
       --base.menu_description,
       base.allergy,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_allergy` ,
                    'menu_allergy_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('Lactose') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10000)
 WHERE distance < .67
GROUP BY ALL
ORDER BY distance;

#### Basic search for a compound term

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- We can do semi-simple search like "lactose intolerance"
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       COUNT(base.menu_id) AS menu_count,
       --base.menu_name
       --base.menu_description,
       --TRIM(base.allergy) as allergy,
       base.allergy,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_allergy` ,
                    'menu_allergy_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('lactose intolerance') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10000)
 WHERE distance < .76
GROUP BY ALL
ORDER BY distance;

#### Searching a complex description of a customers allergies

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Show: Complex allergy information does not return anything.
--
-- The embeddings does not match a complex set of sentances:
--   Severe nut allergy, especially hazelnuts and almonds.
--   Also, avoid any chocolate containing traces of soy lecithin.
--   I have had anaphylactic reactions in the past.
--   Please ensure all preparation surfaces are thoroughly cleaned.
--   I also have a sensitivity to artificial sweeteners, particularly aspartame.
--   I would prefer organic chocolate if possible.
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       COUNT(base.menu_id) AS menu_count,
       --base.menu_name
       --base.menu_description,
       --TRIM(base.allergy) as allergy,
       base.allergy,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_allergy` ,
                    'menu_allergy_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('Severe nut allergy, especially hazelnuts and almonds. Also, avoid any chocolate containing traces of soy lecithin. I have had anaphylactic reactions in the past. Please ensure all preparation surfaces are thoroughly cleaned. I also have a sensitivity to artificial sweeteners, particularly aspartame. I would prefer organic chocolate if possible.') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10000)
 WHERE distance < .76
GROUP BY ALL
ORDER BY distance;

#### Breaking apart a complex description of a customers allergies into indivual allergy items

In [ ]:
%%bigquery customer_df

----------------------------------------------------------------------------------------------------------
-- Simulate an Oracle query by getting customer allergy information
----------------------------------------------------------------------------------------------------------
SELECT *
  FROM `chocolate_ai.oracle_customer`
WHERE customer_id = 1;

In [ ]:
##########################################################################################################
## Extract just the customer food allergies using Gemini
## In production you should double check these results.
## Use Gemini Pro "gemini-2.0-pro-exp-02-05" (not flash), since we want a deeper reasoning on this task.
##########################################################################################################

customer_allergy_information =  customer_df.iloc[0]["allergy_information"]
print(f"customer_allergy_information: {customer_allergy_information}")
customer_allergy_information = customer_allergy_information.lower()

# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "allergies" : "text",
#  }

response_schema = {
  "type": "object",
  "required": [
    "allergies"
  ],
  "properties": {
    "allergies": {
      "type": "string"
    }
  }
}

prompt = """You are an expert in allergies and processing of tokens in sentances.
I need to search vector embeddings for matching food allergies.
I only want items the customer is allergic to.  Do not include their preferences in the result list.
This is for a chocolate company so most items contain chocolate so check twice if the customer said they are allergic to chocolate; otherwise, we would exclude the entire menu.

I need you to create a comma seperated list of words from the below customer preference and follow the below rules.

Rules:
- Preprocessing and Tokenization (with a Focus on Negation):
  - Tokenization: Break the user's sentence into individual words or phrases. This is similar to what we did with the allergy_information, but now we need to be a bit smarter. We can't just split on spaces; we need to handle punctuation and potentially multi-word allergens.
  - Stop Word Removal (Carefully!): Remove common words like "I", "do", "the", "in", "my", etc. However, be extremely careful about removing words related to negation ("not", "no", "without"). These are crucial for understanding the user's intent. We'll handle these in the next step.
  - Stemming/Lemmatization (optional but helpful): Consider stemming (reducing words to their root form, e.g., "colors" -> "color") or lemmatization (finding the dictionary form, e.g., "better" -> "good"). This can help match variations like "sweetener" and "sweeteners". SpaCy is great for this.

-  Negation Handling:
  - This is the most important part of processing this type of query. We need to distinguish between positive ("I want milk") and negative ("I don't want milk") statements about allergens. Here are several strategies, ordered from simpler to more complex:
  - Simple Negation Flag (Recommended for most cases):
  - Iterate through the tokenized sentence.
  - If you encounter a negation word ("not", "no", "without", "avoid", etc.), set a negation_flag to True.
  - When you encounter a known allergen, check the negation_flag.
  - If negation_flag is True, this allergen is excluded (the user doesn't want it).
  - If negation_flag is False, this allergen is included (the user does want it – though this is less common in allergy contexts).
  - Reset the negation_flag to False after processing each allergen or at the end of a clause (e.g., after a comma or "and").

<customer_preference>
{customer_allergy_information}
</customer_preference>

"""

# Use LLM to generate data
llm_response = GeminiLLM(prompt, response_schema=response_schema, model="gemini-2.0-pro-exp-02-05", temperature=.5)

# Parse response (we know the JSON since we passed it to our LLM)
llm_json_response = json.loads(llm_response)
print(json.dumps(llm_json_response, indent=2))
allergies = llm_json_response["allergies"]



#### Searching a complex description using Gemini extract allery keywords

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Take the above comma seperated list of allergies and split and embed each value
-- Then search our menu allergy vector embeddings (all at once)
-- We will end up with menu items that the customer should avoid (distance < .80) or we
-- will end up with menus items that are okay (distance >= .80)
----------------------------------------------------------------------------------------------------------
WITH split_allergies AS
(
  SELECT SPLIT(LOWER("nut, hazelnuts, almonds, soy lecithin, artificial sweeteners, aspartame"), ',') AS allergy_array
),
allergies AS
(
  SELECT DISTINCT allergy
    FROM split_allergies
         JOIN UNNEST(allergy_array) AS allergy
),
customer_vector_embeddings AS
(
SELECT allergy,
       ml_generate_embedding_result AS customer_allergy_embedding
  FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                            (SELECT TRIM(allergy) AS allergy,
                                    TRIM(allergy) AS content
                               FROM allergies),
                             STRUCT(TRUE AS flatten_json_output,
                                    'SEMANTIC_SIMILARITY' as task_type,
                                    768 AS output_dimensionality))
)
SELECT query.allergy as customer_allergy,
       base.menu_id  AS menu_id,
       base.menu_name AS menu_name,
       base.menu_description AS menu_description,
       base.allergy as menu_allergy,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_allergy` ,
                    'menu_allergy_embedding',         -- column in table to search
                    TABLE customer_vector_embeddings, -- source table of multiple embeddings
                    'customer_allergy_embedding',     -- source table of customer allergies embeddings
                    top_k => 1000)
 WHERE distance < .80
ORDER BY distance;

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- Same query as above, but with just with distinct results
----------------------------------------------------------------------------------------------------------
WITH split_allergies AS
(
  SELECT SPLIT(LOWER("hazelnuts, almonds, soy lecithin, artificial sweeteners, aspartame"), ',') AS allergy_array
),
allergies AS
(
  SELECT DISTINCT allergy
    FROM split_allergies
         JOIN UNNEST(allergy_array) AS allergy
),
customer_vector_embeddings AS
(
SELECT allergy,
       ml_generate_embedding_result AS customer_allergy_embedding
  FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                            (SELECT TRIM(allergy) AS allergy,
                                    TRIM(allergy) AS content
                               FROM allergies),
                             STRUCT(TRUE AS flatten_json_output,
                                    'SEMANTIC_SIMILARITY' as task_type,
                                    768 AS output_dimensionality))
)
SELECT DISTINCT query.allergy as customer_allergy,
       --base.menu_id  AS menu_id,
       --base.menu_name AS menu_name,
       --base.menu_description AS menu_description,
       base.allergy as menu_allergy,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_allergy` ,
                    'menu_allergy_embedding',         -- column in table to search
                    TABLE customer_vector_embeddings, -- source table of multiple embeddings
                    'customer_allergy_embedding',     -- source table of customer allergies embeddings
                    top_k => 1000)
 WHERE distance < .65
ORDER BY distance;

### <font color='#4285f4'>Menu (Product) Search with Re-Ranking</font>

#### Basic search for "chocolate truffles" in Menu **Name** and Menu **Description**

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- We can do simple searchs like "chocolate truffles"
-- This is ONLY searching the menu name
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       base.menu_id,
       base.menu_name,
       base.menu_description,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_name` ,
                    'menu_name_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('chocolate truffles') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10)
 --WHERE distance < .67
ORDER BY distance;

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- We can do simple searchs like "chocolate truffles"
-- This is ONLY searching the menu description
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       base.menu_id,
       base.menu_name,
       base.menu_description,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_description` ,
                    'menu_description_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('chocolate truffles') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10)
ORDER BY distance;

#### Combine our Menu **Name** and Menu **Description** search results

In [ ]:
%%bigquery

----------------------------------------------------------------------------------------------------------
-- UNION the menu name and menu description searches into one result
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       base.menu_id,
       base.menu_name,
       base.menu_description,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_name` ,
                    'menu_name_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('chocolate truffles') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10)
UNION ALL
SELECT query.query AS search_string,
       base.menu_id,
       base.menu_name,
       base.menu_description,
       distance
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_description` ,
                    'menu_description_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('chocolate truffles') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10)
ORDER BY distance;

In [ ]:
%%bigquery ranking_dataset

----------------------------------------------------------------------------------------------------------
-- Place the UNION of search results into a dataframe, so we can use it in Python code
----------------------------------------------------------------------------------------------------------
SELECT query.query AS search_string,
       base.menu_id,
       base.menu_name,
       base.menu_description,
       distance,
       'menu_name' AS source,
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_name` ,
                    'menu_name_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('chocolate truffles') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10)
UNION ALL
SELECT query.query AS search_string,
       base.menu_id,
       base.menu_name,
       base.menu_description,
       distance,
       'menu_description' AS source,
  FROM VECTOR_SEARCH(TABLE `chocolate_ai.menu_embeddings_description` ,
                    'menu_description_embedding', -- column in table to search
                    (SELECT ml_generate_embedding_result,
                            content AS query
                       FROM ML.GENERATE_EMBEDDING(MODEL `chocolate_ai.textembedding_model`,
                                                 (SELECT LOWER('chocolate truffles') AS content),
                                                  STRUCT(TRUE AS flatten_json_output,
                                                        'SEMANTIC_SIMILARITY' as task_type,
                                                        768 AS output_dimensionality) -- struct
                     )),
        top_k => 10)
ORDER BY distance;

#### **Rank** our search result across our two seperate search results

In [ ]:
##########################################################################################################
## Build up a list of our query results (id and content)
## The id must be unique so concatenate the source (menu name or menu description) in case we have the same menu id from both queries.
##########################################################################################################

record_list = []
for index, row in ranking_dataset.iterrows():
  menu_id = row["menu_id"]
  source = row["source"]
  menu_name = row["menu_name"].lower()
  menu_description = row["menu_description"].lower()

  if source == "menu_name":
    content = f"{menu_name}"
  else:
    content = f"{menu_description}"

  record =  {
      "id": f"{source}-{menu_id}",
      "content": content
  }
  record_list.append(record)

print(f"record_list: {record_list}")

In [ ]:
##########################################################################################################
## Now we want to rank our menu name and menu description search results.
## We might have menu names that are an exact match and menu descriptions that are an exact match.
## We basically want to interweave our results into a single list with the most relevant items at the top.
## So we can end up with menu names and menu descriptions in as adjacent items
##########################################################################################################
query = "Which of the below items best match 'chocolate truffles'?"

json_result = vertex_ai_reranking(project_id, query, record_list)

#  Print the results (pretty print)
json_formatted_str = json.dumps(json_result, indent=2)
print(json_formatted_str)